# Spark SQL Engine

[ Ref _Learning Spark v2_ book, _Chapter 3_.]

At a programmatic level, Spark SQL allows developers to issue ANSI SQL:2003 compatible queries on structured data with a schema.

The diagram below summarizes the structure of Spark SQL and its usage:

<img src="https://raw.githubusercontent.com/ZacksAmber/PicGo/master/img/20220228173407.png" width="500">

Ref: [Learning Spark, Zacks Blog](https://zacks.one/learningspark/#Spark-SQL-and-DataFrames-Introduction-to-Built-in-Data-Sources)

Spark SQL allows to read data from structured file formats like the ones at the bottom of the stack (JSON, csv, etc). It then allows to access such data via ODBC connectors or SQL Spark shells by storing this data to temporarily table.

How can I actually run SQL commands in my notebook? We can define a `DataFrame` and run the `createOrReplaceTempView` method. It allows to run SQL queries programmatically and returns the result again as a `DataFrame`.

In [0]:
sf_fire_file = "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"
df = spark.read.csv(sf_fire_file, header=True)

df.createOrReplaceTempView("firecalls")

Now, we have registered a temporary view called `firecalls` that will last for the duration of this session.

 To issue any SQL query, use the `sql()` method on the `SparkSession` instance, `spark`, such as `spark.sql("SELECT * FROM firecalls")`. All `spark.sql` queries executed in this manner return a `DataFrame`.

In [0]:
sql_result_df = spark.sql("""
    SELECT CallType, count(*) as count
    FROM firecalls
    GROUP BY CallType
    ORDER BY count(*) desc
""")
display(sql_result_df)

CallType,count
Medical Incident,2843475
Structure Fire,578998
Alarms,483518
Traffic Collision,175507
Citizen Assist / Service Call,65360
Other,56961
Outside Fire,51603
Vehicle Fire,20939
Water Rescue,20037
Gas Leak (Natural and LP Gases),17284


As we've just seen, the same transformations can be expressed via SQL or DataFrame APIs and still get the same result.

In Databricks, you can convert a cell to a SQL cell by starting it with a `%sql`

In [0]:
%sql
SELECT CallType, count(*) as count
FROM firecalls
GROUP BY CallType
ORDER BY count(*) desc

CallType,count
Medical Incident,2843475
Structure Fire,578998
Alarms,483518
Traffic Collision,175507
Citizen Assist / Service Call,65360
Other,56961
Outside Fire,51603
Vehicle Fire,20939
Water Rescue,20037
Gas Leak (Natural and LP Gases),17284


## Catalyst optimizer

The Catalyst optimizer sits at the core of the Spark SQL engine. It takes a query and converts it to an execution plan. The plan goes through four **transformational phases**:

1. **analysis**. The Spark SQL engine begins by generating an abstract syntax tree (AST) for the SQL or DataFrame query. In this initial phase, any columns or table names will be resolved by consulting an internal **Catalog**, a programmatic interface to Spark SQL that holds a list of names of columns, data types, functions, tables, databases, etc.
2. **logical optimization**. Applying a standardrule based optimization approach, the Catalyst optimizer will first construct a set of multiple plans and then, using its cost-based optimizer (CBO), assign costs to each plan.
3. **physical planning**. Spark SQL generates an optimal physical plan for the selected logical plan, using physical operators that match those available in the Spark execution engine.
4. **code generation**. Generating efficient Java bytecode to run on each machine.

The image below summarazies these phases.

![Four phases of Spark plan](https://www.databricks.com/wp-content/uploads/2018/05/Catalyst-Optimizer-diagram.png)

Ref: [Databricks](https://www.databricks.com/glossary/catalyst-optimizer)

Can we see the plan of our transformations? Yes, we can print it from any `DataFrame` via the `explain` method.

In [0]:
from pyspark.sql.functions import count

count_df = (
    df.select("CallType", "Call Number")
    .groupBy("CallType")
    .count()
    .orderBy("count", ascending=False)
)
display(count_df)

CallType,count
Medical Incident,2843475
Structure Fire,578998
Alarms,483518
Traffic Collision,175507
Citizen Assist / Service Call,65360
Other,56961
Outside Fire,51603
Vehicle Fire,20939
Water Rescue,20037
Gas Leak (Natural and LP Gases),17284


In [0]:
count_df.explain(extended=True)

== Parsed Logical Plan ==
'Sort ['count DESC NULLS LAST], true
+- Aggregate [CallType#26], [CallType#26, count(1) AS count#145L]
   +- Project [CallType#26, Call Number#23]
      +- Relation [Call Number#23,Unit ID#24,Incident Number#25,CallType#26,Call Date#27,Watch Date#28,Call Final Disposition#29,Available DtTm#30,Address#31,City#32,Zipcode of Incident#33,Battalion#34,Station Area#35,Box#36,OrigPriority#37,Priority#38,Final Priority#39,ALS Unit#40,Call Type Group#41,NumAlarms#42,UnitType#43,Unit sequence in call dispatch#44,Fire Prevention District#45,Supervisor District#46,... 4 more fields] csv

== Analyzed Logical Plan ==
CallType: string, count: bigint
Sort [count#145L DESC NULLS LAST], true
+- Aggregate [CallType#26], [CallType#26, count(1) AS count#145L]
   +- Project [CallType#26, Call Number#23]
      +- Relation [Call Number#23,Unit ID#24,Incident Number#25,CallType#26,Call Date#27,Watch Date#28,Call Final Disposition#29,Available DtTm#30,Address#31,City#32,Zipcode of Inci

How do we read this? It should be read bottom-up. So, looking at the _Parsed Logical Plan_:

1. Relation [ ... ] csv (reading the csv data source)
2. Project `CallType` and `CallNumber` (`CallNumber` will disappear from _Optimized Logical Plan_!)
3. Aggregate (aggregating by `CallType` and counting)

Is the plan behind the PySpark transformations of `count_df` the same as the `SQL` transformation behind `sql_result_df`?

In [0]:
sql_result_df.explain(extended=True)

== Parsed Logical Plan ==
'Sort ['count(1) DESC NULLS LAST], true
+- 'Aggregate ['CallType], ['CallType, 'count(1) AS count#79]
   +- 'UnresolvedRelation [firecalls], [], false

== Analyzed Logical Plan ==
CallType: string, count: bigint
Sort [count#79L DESC NULLS LAST], true
+- Aggregate [CallType#26], [CallType#26, count(1) AS count#79L]
   +- SubqueryAlias firecalls
      +- View (`firecalls`, [Call Number#23,Unit ID#24,Incident Number#25,CallType#26,Call Date#27,Watch Date#28,Call Final Disposition#29,Available DtTm#30,Address#31,City#32,Zipcode of Incident#33,Battalion#34,Station Area#35,Box#36,OrigPriority#37,Priority#38,Final Priority#39,ALS Unit#40,Call Type Group#41,NumAlarms#42,UnitType#43,Unit sequence in call dispatch#44,Fire Prevention District#45,Supervisor District#46,Neighborhood#47,Location#48,RowID#49,Delay#50])
         +- Relation [Call Number#23,Unit ID#24,Incident Number#25,CallType#26,Call Date#27,Watch Date#28,Call Final Disposition#29,Available DtTm#30,Address#

The _Optimized Logical Plan_ look actually the same for the transformation in SQL and in PySpark! That is, regardless of the language you use, your computation undergoes the same journey and the resulting bytecode is likely the same.

## SQL Tables and Views

[ Ref _Learning Spark v2_ book, _Chapter 4_.]

Tables hold data. Associated with each table in Spark is its relevant metadata, which is information about the table and its data: the schema, description, table name, database name, column names, partitions, physical location where the actual data resides, etc. All of this is stored in a **central metastore**. Spark by default uses the Apache Hive metastore, located at /user/hive/warehouse, to persist all the metadata about your tables. However, you may change the default location.

Spark can handle two types of SQL tables: _managed_ and _unmanaged_.

| Table Type | Metadata | Data | Commands |
|-|-|-|-|
| Managed | Spark manages the metadata* | Spark manages the data* | Spark manages everything about the table, eg `DELETE` deletes both metadata and data |
| Unmanaged | Spark manages the metadata* | You can manage data yourself in an external data source (eg Cassandra) | Spark manages only the metadata, eg `DELETE` will delete metadata only |

\* eg local filesystem, Azure Blob, AWS S3, etc

In the remaining of this section, we will focus on _managed_ tables only. Let's start by creating our first table. By default, Spark uses a database called `default`. You can of course create and use other databases.

In [0]:
#dbutils.fs.rm('/user/hive/warehouse/managed_us_delay_flights_tbl/', recurse=True)

res2: Boolean = true

In [0]:
spark.sql(
    """CREATE TABLE managed_us_delay_flights_tbl (
        date STRING,
        delay INT,
        distance INT,
        origin STRING,
        destination STRING
    )"""
)

Out[7]: DataFrame[]

Alternatively we can use the DataFrame APIs via `saveAsTable`.

In [0]:
%sql DROP TABLE managed_us_delay_flights_tbl

In [0]:
csv_file = "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("date", StringType(), False),
    StructField("delay", IntegerType(), False),
    StructField("distance", IntegerType(), False),
    StructField("origin", StringType(), False),
    StructField("destination", StringType(), False)
])

flights_df = spark.read.csv(csv_file, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

In [0]:
%sql
SELECT *
FROM managed_us_delay_flights_tbl
LIMIT 10

date,delay,distance,origin,destination
date,null,null,origin,destination
01011245,6,602,ABE,ATL
01020600,-8,369,ABE,DTW
01021245,-2,602,ABE,ATL
01020605,-4,602,ABE,ATL
01031245,-4,602,ABE,ATL
01030605,0,602,ABE,ATL
01041243,10,602,ABE,ATL
01040605,28,602,ABE,ATL
01051245,88,602,ABE,ATL


**Views** can be global (visible across all `SparkSessions` on a given cluster) or session-scoped (visible only to a single `SparkSession`), and they are temporary: they disappear after your Spark application terminates.

In [0]:
flights_df.where(flights_df.origin == "SFO")

# temporary view
flights_df.createOrReplaceTempView("sfo_flights_origins_temporary")

# global temporary
flights_df.createOrReplaceGlobalTempView("sfo_flights_origing_global_temporary")
# when accessing a global temporary view you must use the prefix global_temp.<view_name>,

You can view the **metadata** of your SQL tables and databases. These metadata are stored in an abstraction called `Catalog` and can be accessed with the following methods.

In [0]:
print(spark.catalog.listDatabases())
print(spark.catalog.listTables())

[Database(name='default', description='Default Hive database', locationUri='dbfs:/user/hive/warehouse')]
[Table(name='managed_us_delay_flights_tbl', database='default', description=None, tableType='MANAGED', isTemporary=False), Table(name='firecalls', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='sfo_flights_origins_temporary', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


Reading data from a SQL table into a `DataFrame` can be done by typing a query or by `spark.table` method.

In [0]:
flights_df = spark.sql("SELECT * FROM managed_us_delay_flights_tbl")
# or
flights_df = spark.table("managed_us_delay_flights_tbl")

## Data sources

You can read and write data from/to built-in data sources (eg Parquet, JSON, etc) via Spark Data Source APIs. The APIs offer the `DataFrameReader` and `DataFrameWriter` constructs.

### DataFrameReader

It has a defined format and a recommended pattern for usage:

```
DataFrameReader.format(args).option("key", "value").schema(args).load()
```

Let's go through these objects

- `DataFremeReader` accessible via a `SparkSession` instance, eg `spark.read`
- `format` specifies the format of the data source, eg `"json"`. Default `"parquet"`
- `option` a series of key-value pairs of options, eg `("inferSchema", True)`
- `schema` you can provide the schema here
- `load` path to data source, eg `"/path/to/data/source.csv"`

Let's look at an example with **Parquet** files.

In [0]:
%fs ls /databricks-datasets/learning-spark-v2/flights/summary-data/parquet/

path,name,size,modificationTime
dbfs:/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/,2010-summary.parquet/,0,0


In [0]:
file = """/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"""
df = spark.read.format("parquet").load(file)

display(df)

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


### DataFrameWriter
`DataFrameWriter` does the reverse of its counterpart: it saves or writes data to a speci‐ fied built-in data source. The common usage pattern

```
DataFrameWriter
  .format(args)
  .mode(args)
  .option(args)
  .partitionBy(args)
  .save(path)
```

Let's go through these objects

- `DataFremeWriter` accessible via a `DataFrame` instance, eg `df.write`
- `format` specifies the format of the data source, eg `"json"`. Default `"parquet"`
- `mode` specifies the behavior when data or table already exists, eg `"overwrite"`
- `option` a series of key-value pairs of options, eg `("inferSchema", True)`
- `partitionBy` partitions the output by the given columns (eg `"destination"`) on the file system.
- `load` path to data source, eg `"/path/to/data/source.csv"`

In [0]:
(
    flights_df.write
    .format("parquet")
    .mode("overwrite")
    .option("compression", "snappy")
    .partitionBy("destination")
    .save("/tmp/data/parquet/df_parquet")
)

In [0]:
%fs ls /tmp/data/parquet/df_parquet

path,name,size,modificationTime
dbfs:/tmp/data/parquet/df_parquet/_SUCCESS,_SUCCESS,0,1660629221000
dbfs:/tmp/data/parquet/df_parquet/destination=ABE/,destination=ABE/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ABI/,destination=ABI/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ABQ/,destination=ABQ/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ABR/,destination=ABR/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ABY/,destination=ABY/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ACT/,destination=ACT/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ACV/,destination=ACV/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ADK/,destination=ADK/,0,0
dbfs:/tmp/data/parquet/df_parquet/destination=ADQ/,destination=ADQ/,0,0
